In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/2.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：93



,No,Url
0,1,https://www.dormanproducts.com/p-17668-34198.a...
1,2,https://www.dormanproducts.com/p-7395-34197.as...
2,3,https://www.dormanproducts.com/p-6889-34187.as...
3,4,https://www.dormanproducts.com/p-25955-34188.a...
4,5,https://www.dormanproducts.com/p-32262-34189.a...
...,...,...
88,89,https://www.dormanproducts.com/p-131747-34869....
89,90,https://www.dormanproducts.com/p-131983-34868....
90,91,https://www.dormanproducts.com/p-88995-34283.a...
91,92,https://www.dormanproducts.com/p-81798-419-500...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.dormanproducts.com/p-17668-34198.a...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
            if type_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

type_

1


'Air Conditioning Bypass Pulley'

In [6]:
dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
for href in list_href:
    if href.startswith('http'):
        dict_src[str(i)] = href
        i += 1

if not list_href:
    list_href = [href.strip() for href in html.xpath('//a[@id="ProductPic"]/@href')]
    if list_href:
        dict_src = {'0': list_href[0]}

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/34198-007.JPG'}

In [7]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'34198'

In [8]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Air Conditioning Bypass Pulley'

In [9]:
list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'Buick 1991 & 1989, Chevrolet 1996-94, 1991-89, GMC 1996-94, Oldsmobile 1991-89, Pontiac 1991-89'

In [10]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - HELP! - Boxed'

In [11]:
value = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'https://www.dormanproducts.com/{value[0].strip()}' if value else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=17668&PartType=Air%20Conditioning%20Bypass%20Pulley'

In [12]:
list_tr, dict_count = html.xpath('//section[@id="productSpec"]/div/table/tr'), {}
for tr in list_tr:
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specification, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_no[key]}': tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Compressor Type': 'Fs10'},
 '1': {'Installation Instructions Included': 'Yes'},
 '2': {'Material': 'Plastic Pulley; Aluminum Bracket'},
 '3': {'Mounting Bracket Included': 'Yes'},
 '4': {'Mounting Hardware Included': 'Yes'},
 '5': {'Number of Pulleys Included': '1'},
 '6': {'Outside Diameter': '114 mm'},
 '7': {'Package Contents': 'Pulley; Bracket; and Installation Hardware'},
 '8': {'Package Quantity': '1'},
 '9': {'Packaging Type': 'Box'},
 '10': {'Pulley Type/Grooves': 'Serpentine'}}

In [13]:
list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

oem

''

In [14]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Type': type_,
                          'Part Number': part_number,
                          'Brand': brand,
                          'Description': description,
                          'Application Summary': application_summary,
                          'OEM': oem,
                          'Picture': '',
                          'Url': url_request,
                          'Json_Src': json.dumps(dict_src),
                          'Vehicle Url': vehicle_url,
                          'Json_Specification': json.dumps(dict_specification)})

list_dict_correct

[{'No': 1,
  'Type': 'Air Conditioning Bypass Pulley',
  'Part Number': '34198',
  'Brand': 'Dorman - HELP! - Boxed',
  'Description': 'Air Conditioning Bypass Pulley',
  'Application Summary': 'Buick 1991 & 1989, Chevrolet 1996-94, 1991-89, GMC 1996-94, Oldsmobile 1991-89, Pontiac 1991-89',
  'OEM': '',
  'Picture': '',
  'Url': 'https://www.dormanproducts.com/p-17668-34198.aspx?origin=keyword',
  'Json_Src': '{"0": "https://static.dormanproducts.com/images/product/medium/34198-007.JPG"}',
  'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=17668&PartType=Air%20Conditioning%20Bypass%20Pulley',
  'Json_Specification': '{"0": {"Compressor Type": "Fs10"}, "1": {"Installation Instructions Included": "Yes"}, "2": {"Material": "Plastic Pulley; Aluminum Bracket"}, "3": {"Mounting Bracket Included": "Yes"}, "4": {"Mounting Hardware Included": "Yes"}, "5": {"Number of Pulleys Included": "1"}, "6": {"Outside Diameter": "114 mm"}, "7": {"Package Contents": "Pulley; Brac

In [15]:
crawler_status = True

crawler_status

True

In [16]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/p-17668-34198.a...
 Request_Url    https://www.dormanproducts.com/p-17668-34198.a...
 Name: 0, dtype: object]

In [17]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Type', 'Part Number'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct['No'] = range(1, len(df_correct) + 1)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Air Conditioning Bypass Pulley,34198,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Buick 1991 & 1989, Chevrolet 1996-94, 1991-89,...",,,https://www.dormanproducts.com/p-17668-34198.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Compressor Type"": ""Fs10""}, ""1"": {""Inst..."


In [18]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.dormanproducts.com/p-17668-34198.a...,https://www.dormanproducts.com/p-17668-34198.a...
